In [63]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans
import numpy as np
from pandas.io.json import json_normalize
import airbnb
import ezhc as hc


#### Fetch the latitude & longitude of the cities compared

In [15]:
geolocator = Nominatim(user_agent="blabla_explorer")

In [16]:
address = 'Bayonne'
location = geolocator.geocode(address)
latitude_bayonne = location.latitude
longitude_bayonne = location.longitude
print('The geograpical coordinate of Bayonne are {}, {}.'.format(latitude_bayonne, longitude_bayonne))

The geograpical coordinate of Bayonne are 43.4933379, -1.475099.


In [17]:
address = 'Hossegor'
location = geolocator.geocode(address)
latitude_hossegor = location.latitude
longitude_hossegor = location.longitude
print('The geograpical coordinate of Hossegor are {}, {}.'.format(latitude_hossegor, longitude_hossegor))

The geograpical coordinate of Hossegor are 43.6591945, -1.42712819513575.


In [18]:
address = 'Biarritz'
location = geolocator.geocode(address)
latitude_biarritz = location.latitude
longitude_biarritz = location.longitude
print('The geograpical coordinate of Biarritz are {}, {}.'.format(latitude_biarritz, longitude_biarritz))

The geograpical coordinate of Biarritz are 43.4832523, -1.5592776.


In [19]:
address = 'Bordeaux'
location = geolocator.geocode(address)
latitude_bordeaux = location.latitude
longitude_bordeaux = location.longitude
print('The geograpical coordinate of Bordeaux are {}, {}.'.format(latitude_bordeaux, longitude_bordeaux))

The geograpical coordinate of Bordeaux are 44.841225, -0.5800364.


In [20]:
address = 'Anglet'
location = geolocator.geocode(address)
latitude_anglet= location.latitude
longitude_anglet = location.longitude
print('The geograpical coordinate of Anglet are {}, {}.'.format(latitude_anglet, longitude_anglet))

The geograpical coordinate of Anglet are 43.4813927, -1.5149935.


In [21]:
address = 'Paris'
location = geolocator.geocode(address)
latitude_paris = location.latitude
longitude_paris = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude_paris, longitude_paris))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [22]:
address = 'Nantes'
location = geolocator.geocode(address)
latitude_nantes = location.latitude
longitude_nantes = location.longitude
print('The geograpical coordinate of Nantes are {}, {}.'.format(latitude_nantes, longitude_nantes))

The geograpical coordinate of Nantes are 47.2186371, -1.5541362.


In [23]:
address = 'Montpellier'
location = geolocator.geocode(address)
latitude_montpellier = location.latitude
longitude_montpellier = location.longitude
print('The geograpical coordinate of Montpellier are {}, {}.'.format(latitude_montpellier, longitude_montpellier))

The geograpical coordinate of Montpellier are 43.6112422, 3.8767337.


In [24]:
address = 'Toulouse'
location = geolocator.geocode(address)
latitude_toulouse = location.latitude
longitude_toulouse = location.longitude
print('The geograpical coordinate of Toulouse are {}, {}.'.format(latitude_toulouse, longitude_toulouse))

The geograpical coordinate of Toulouse are 43.6044622, 1.4442469.


#### Get the average price on Airbnb for both cities

In [25]:
username = open("C:\\temp\\airbnb_email", "r").read()
password = open("C:\\temp\\airbnb_password", "r").read()

In [26]:
api = airbnb.Api(username, password)

VerificationError: 

In [27]:
homes_hossegor = api.get_homes(gps_lat=latitude_hossegor, gps_lng=longitude_hossegor)

NameError: name 'api' is not defined

In [28]:
avg_price_hossegor = homes_hossegor['explore_tabs'][0]['home_tab_metadata']['price_histogram']['average_price']
print("The average price of a night in an Airbnb in Hossegor is {}€".format(avg_price_hossegor))

NameError: name 'homes_hossegor' is not defined

In [29]:
homes_bayonne = api.get_homes(gps_lat=latitude_bayonne, gps_lng=longitude_bayonne)

NameError: name 'api' is not defined

In [30]:
avg_price_bayonne = homes_bayonne['explore_tabs'][0]['home_tab_metadata']['price_histogram']['average_price']
print("The average price of a night in an Airbnb in Bayonne is {}€".format(avg_price_bayonne))

NameError: name 'homes_bayonne' is not defined

#### Get the venues for both cities

In [31]:
CLIENT_ID = open("C:\\temp\\foursquare_id", "r").read()
CLIENT_SECRET = open("C:\\temp\\foursquare_key", "r").read()
VERSION = '20180605' # Foursquare API version

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            500)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude',
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
all_venues = getNearbyVenues(names=['Hossegor', 'Bayonne', 'Biarritz', 'Paris', 'Anglet', 'Nantes', 'Montpellier', 'Toulouse', 'Bordeaux'], 
                            latitudes=[latitude_hossegor, latitude_bayonne, latitude_biarritz, latitude_paris, latitude_anglet, latitude_nantes, latitude_montpellier, latitude_toulouse, latitude_bordeaux], 
                            longitudes=[longitude_hossegor, longitude_bayonne, longitude_biarritz, longitude_paris, longitude_anglet, longitude_nantes, longitude_montpellier, longitude_toulouse, longitude_bordeaux], 
)

Hossegor
Bayonne
Biarritz
Paris
Anglet
Nantes
Montpellier
Toulouse
Bordeaux


In [34]:
all_venues

,Neighborhood,Latitude,Longitude,Venue,Venue Category
0,Hossegor,43.659194,-1.427128,Marché d'Hossegor,Farmers Market
1,Hossegor,43.659194,-1.427128,Waxed,Café
2,Hossegor,43.659194,-1.427128,glaces Romane,Ice Cream Shop
3,Hossegor,43.659194,-1.427128,L'Etiquette,Wine Bar
4,Hossegor,43.659194,-1.427128,Touring Café,Café
...,...,...,...,...,...
560,Bordeaux,44.841225,-0.580036,Donuts Coffee,Coffee Shop
561,Bordeaux,44.841225,-0.580036,noorn akorn,Asian Restaurant
562,Bordeaux,44.841225,-0.580036,Le Pizzaïolo,Pizza Place
563,Bordeaux,44.841225,-0.580036,Le Rohan,Café


In [41]:
df = all_venues.groupby(['Neighborhood', 'Venue Category']).count()
df

Latitude  Longitude  Venue
Neighborhood Venue Category                                           
Anglet       Farmers Market                        1          1      1
             Gastropub                             1          1      1
             Insurance Office                      1          1      1
             Salon / Barbershop                    1          1      1
Bayonne      Bus Stop                              1          1      1
...                                              ...        ...    ...
Toulouse     Steakhouse                            1          1      1
             Tea Room                              4          4      4
             Theater                               1          1      1
             Vegetarian / Vegan Restaurant         1          1      1
             Wine Bar                              1          1      1

[297 rows x 3 columns]

In [62]:
colors_2 = ['#7cb5ec', '#434348', '#90ed7d', '#f7a35c', '#8085e9',
          '#f15c80', '#e4d354', '#2b908f', '#f45b5b', '#91e8e1']

points = hc.build.series_tree(df[['Venue']], set_color=True, colors=colors_2, set_value=True, precision=2)
points[:5]

[{'name': 'Farmers Market', 'value': 1, 'id': '0.0.0', 'parent': '0.0'},
 {'name': 'Gastropub', 'value': 1, 'id': '0.0.1', 'parent': '0.0'},
 {'name': 'Insurance Office', 'value': 1, 'id': '0.0.2', 'parent': '0.0'},
 {'name': 'Salon / Barbershop', 'value': 1, 'id': '0.0.3', 'parent': '0.0'},
 {'name': 'Anglet', 'id': '0.0', 'color': '#7cb5ec', 'value': 4}]

In [47]:
g = hc.Highcharts()

g.chart.type = 'treemap'
g.chart.width = 900
g.chart.height = 600

g.title.text = 'Venues per city and category'
g.subtitle.text = 'Click points to drill down'
g.exporting = False

g.series = [{
    'type': "treemap",
    'layoutAlgorithm': 'squarified',
    'allowDrillToNode': True,
    'dataLabels': {
        'enabled': False
    },
    'levelIsConstant': False,
    'levels': [{
        'level': 1,
        'dataLabels': {
            'enabled': True
        },
        'borderWidth': 3
    }],
    'data': points,
}]

g.plot(version='7.0.3')

In [48]:
all_venues.head()

,Neighborhood,Latitude,Longitude,Venue,Venue Category
0,Hossegor,43.659194,-1.427128,Marché d'Hossegor,Farmers Market
1,Hossegor,43.659194,-1.427128,Waxed,Café
2,Hossegor,43.659194,-1.427128,glaces Romane,Ice Cream Shop
3,Hossegor,43.659194,-1.427128,L'Etiquette,Wine Bar
4,Hossegor,43.659194,-1.427128,Touring Café,Café


In [49]:
df_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

df_onehot['Neighborhood'] = all_venues['Neighborhood'] 

fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Neighborhood,Alsatian Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Bagel Shop,...,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Toy / Game Store,Tram Station,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Women's Store
0,Hossegor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hossegor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hossegor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hossegor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Hossegor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Alsatian Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auvergne Restaurant,Bagel Shop,...,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Toy / Game Store,Tram Station,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Women's Store
0,Anglet,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Bayonne,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Biarritz,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.00
3,Bordeaux,0.00,0.010638,0.000000,0.00,0.021277,0.00,0.021277,0.00,0.000000,...,0.010638,0.000000,0.000000,0.000000,0.010638,0.042553,0.000000,0.000000,0.010638,0.00
4,Hossegor,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00
5,Montpellier,0.00,0.000000,0.010000,0.01,0.020000,0.00,0.000000,0.00,0.020000,...,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.040000,0.00
6,Nantes,0.00,0.000000,0.000000,0.00,0.015873,0.00,0.000000,0.00,0.015873,...,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Paris,0.01,0.000000,0.000000,0.05,0.010000,0.01,0.000000,0.01,0.000000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01
8,Toulouse,0.00,0.000000,0.027397,0.00,0.013699,0.00,0.000000,0.00,0.000000,...,0.000000,0.013699,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.013699,0.00


In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anglet,Salon / Barbershop,Insurance Office,Farmers Market,Gastropub,Department Store,Fast Food Restaurant,Electronics Store,Diner,Dessert Shop,Women's Store
1,Bayonne,French Restaurant,Hotel,Tapas Restaurant,Restaurant,Supermarket,Market,Diner,Pub,Creperie,Plaza
2,Biarritz,French Restaurant,Hotel,Tapas Restaurant,Bar,Café,Restaurant,Nightclub,Dessert Shop,Spanish Restaurant,Seafood Restaurant
3,Bordeaux,French Restaurant,Coffee Shop,Plaza,Pedestrian Plaza,Hotel,Shopping Mall,Tram Station,Bistro,Multiplex,Tea Room
4,Hossegor,Café,Wine Bar,Diner,Brewery,Sandwich Place,Farmers Market,Electronics Store,Bar,Bakery,Golf Course


In [53]:
grouped_clustering

NameError: name 'grouped_clustering' is not defined

In [54]:
# set number of clusters
kclusters = 5

grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

clusters = pd.DataFrame(kmeans.labels_[0:10], columns=['cluster'])

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([1, 3, 3, 0, 2, 4, 4, 0, 3])

In [55]:
clusters

,cluster
0,1
1,3
2,3
3,0
4,2
5,4
6,4
7,0
8,3


In [56]:
res = pd.concat([neighborhoods_venues_sorted, clusters], axis=1)
res = pd.merge(left=res, right=all_venues, on='Neighborhood')
res = res[['Latitude', 'Longitude', 'Neighborhood', 'cluster']]
res

,Latitude,Longitude,Neighborhood,cluster
0,43.481393,-1.514994,Anglet,1
1,43.481393,-1.514994,Anglet,1
2,43.481393,-1.514994,Anglet,1
3,43.481393,-1.514994,Anglet,1
4,43.493338,-1.475099,Bayonne,3
...,...,...,...,...
560,43.604462,1.444247,Toulouse,3
561,43.604462,1.444247,Toulouse,3
562,43.604462,1.444247,Toulouse,3
563,43.604462,1.444247,Toulouse,3


In [70]:
# create map
map_clusters = folium.Map(location=[43.6044622, 1.4442469], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(res['Latitude'], res['Longitude'], res['Neighborhood'], res['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters